<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Contextual-Multi-Armed-Bandits-with-Constraints" data-toc-modified-id="Contextual-Multi-Armed-Bandits-with-Constraints-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Contextual Multi Armed Bandits with Constraints</a></span><ul class="toc-item"><li><span><a href="#Matching-patients-to-providers" data-toc-modified-id="Matching-patients-to-providers-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Matching patients to providers</a></span></li><li><span><a href="#Satisfying-Constraints-with-a-Perfect-Oracle" data-toc-modified-id="Satisfying-Constraints-with-a-Perfect-Oracle-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Satisfying Constraints with a Perfect Oracle</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Example:" data-toc-modified-id="Example:-1.2.0.1"><span class="toc-item-num">1.2.0.1&nbsp;&nbsp;</span>Example:</a></span></li></ul></li><li><span><a href="#Linear-Optimization-without-Assignment" data-toc-modified-id="Linear-Optimization-without-Assignment-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Linear Optimization without Assignment</a></span></li><li><span><a href="#Formulation" data-toc-modified-id="Formulation-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Formulation</a></span><ul class="toc-item"><li><span><a href="#Classic-Binary-Optimization" data-toc-modified-id="Classic-Binary-Optimization-1.2.2.1"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>Classic Binary Optimization</a></span></li><li><span><a href="#Linear-optimization-w/o-assignment" data-toc-modified-id="Linear-optimization-w/o-assignment-1.2.2.2"><span class="toc-item-num">1.2.2.2&nbsp;&nbsp;</span>Linear optimization w/o assignment</a></span></li><li><span><a href="#Convert-quadratic-to-linear-using-a-trick-(Mccormick-Envelopes?)" data-toc-modified-id="Convert-quadratic-to-linear-using-a-trick-(Mccormick-Envelopes?)-1.2.2.3"><span class="toc-item-num">1.2.2.3&nbsp;&nbsp;</span>Convert quadratic to linear using a trick (Mccormick Envelopes?)</a></span></li><li><span><a href="#Final-Formulation" data-toc-modified-id="Final-Formulation-1.2.2.4"><span class="toc-item-num">1.2.2.4&nbsp;&nbsp;</span>Final Formulation</a></span></li></ul></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Simulation</a></span><ul class="toc-item"><li><span><a href="#Generate-dummy-data" data-toc-modified-id="Generate-dummy-data-1.2.3.1"><span class="toc-item-num">1.2.3.1&nbsp;&nbsp;</span>Generate dummy data</a></span></li><li><span><a href="#Dummy-Data-with-maximums-and-minimums" data-toc-modified-id="Dummy-Data-with-maximums-and-minimums-1.2.3.2"><span class="toc-item-num">1.2.3.2&nbsp;&nbsp;</span>Dummy Data with maximums and minimums</a></span></li><li><span><a href="#Optimization-function" data-toc-modified-id="Optimization-function-1.2.3.3"><span class="toc-item-num">1.2.3.3&nbsp;&nbsp;</span>Optimization function</a></span></li><li><span><a href="#Run-Simulation" data-toc-modified-id="Run-Simulation-1.2.3.4"><span class="toc-item-num">1.2.3.4&nbsp;&nbsp;</span>Run Simulation</a></span></li></ul></li><li><span><a href="#Lift-over-greedy-algorithm" data-toc-modified-id="Lift-over-greedy-algorithm-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Lift over greedy algorithm</a></span><ul class="toc-item"><li><span><a href="#Expected-number-of-appointments-of-the-linear-optimizer" data-toc-modified-id="Expected-number-of-appointments-of-the-linear-optimizer-1.2.4.1"><span class="toc-item-num">1.2.4.1&nbsp;&nbsp;</span>Expected number of appointments of the linear optimizer</a></span></li><li><span><a href="#Expected-number-of-appointments-of-greedy-algorithm" data-toc-modified-id="Expected-number-of-appointments-of-greedy-algorithm-1.2.4.2"><span class="toc-item-num">1.2.4.2&nbsp;&nbsp;</span>Expected number of appointments of greedy algorithm</a></span></li></ul></li><li><span><a href="#Test-on-unseen-data-with-noise" data-toc-modified-id="Test-on-unseen-data-with-noise-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Test on unseen data with noise</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.2.6"><span class="toc-item-num">1.2.6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li></ul></li></ul></div>

# Contextual Multi Armed Bandits with Constraints

Multi-armed bandits (MAB) is a classic formulation of the exploration versus exploitation problem - 
It is a hypothetical experiment where a bandit must choose between multiple slot machines,
each with an unknown payout. More background on the problem can be found here: [1](https://en.wikipedia.org/wiki/Multi-armed_bandit), [2](https://lilianweng.github.io/lil-log/2018/01/23/the-multi-armed-bandit-problem-and-its-solutions.html)

Contextual MAB is an extension of this approach where we factor in the bandit’s environment, or context, when choosing an action. For the scope of this problem, we are going to reduce the contextual bandit problem into a supervised learning problem, where an "oracle" takes into account the context and gives us a score that indicates the probability of success on the next action.



## Matching Patients to Providers

One real-world application of contextual MAB problem in healthcare is sending patient referrals to a network of mental health providers (MHPs). Let's consider a patient referral system that matches a new patient with the "best" available MHP. The system does not have access to real-time availability of the MHP, but it knows the number of new patients the MHP can generally take week over week. This is the maximum number of referrals a MHP can have (1). Similarly, the MHP expects a minimum number of referrals every week, to keep them engaged on the system, and that sets a minimum constraint on the number of referrals (2). 
When a patient is considered for a provider, there is a lot of information that could be used to make a successful match. Patients have conditions that they seek treatments for, and they might have preferences on gender, race, age, specialization etc of the provider. Similarly, the system has data on the provider's history in declining and scheduling referrals. This information containing patient's preferences and provider's history, constitutes the "context" for the problem (3). 
All three of these together make this use-case equivalent to a contextual MAB problem with minimum and maximum constraints:
- A patient referral can be thought of as "playing" a machine; similarly, MHPs are slot machines. 
- The payout of a slot machine is equivalent to the inherent "scheduling rate" of a MHP. Scheduling rate of a MHP is the proportion of the received referrals scheduled for an appointment.
- The inherent scheduling rate varies over time, but we will assume that it remains nearly the same if we manage to honor the maximum and minimum constraints. A MHP will remain happy if they get a minimum number of referrals each week, and not more than the maximum they want. If we don't honor the constraints, the MHP might become disengaged and stop accepting the referrals
- New providers are on-boarded on the system often, which makes exploration a consistent part of the system. 
- Even though MHPs have an "inherent" scheduling rate, it is common for their rate to vary over time due to staffing changes, seasonal variation, business changes etc. 

Unlike the goal of a typical MAB problem, where the goal is to lower "regret", or the difference between an optimal solution and your solution, this system's goal is to optimize the total number of appointments scheduled by the system, across all MHPs.

Referrals are sent out in real-time, which means we cannot do a batch assignment, and that makes this problem tricky.

## Satisfying Constraints with a Perfect Oracle

For the scope of this solution, we are only going to focus on the constraint satisfaction problem. For simplicity, we assume an oracle that accurately predicts the likelihood of success if a patient is sent to a MHP. This oracle, in practice, is a well-calibrated supervised learning algorithm that predicts the odds that a referral will get scheduled if we send it to a given MHP. 

This formulation violates the standard MAB problem– where the focus is on minimizing regret. One might argue that this is not a classic MAB problem anymore;
Before implementing this solution, we used the classic solution for the MAB problem of Thompson sampling [1](https://en.wikipedia.org/wiki/Thompson_sampling) that minimizes regret. However, we realized that minimizing regret is not an issue at all. On the contrary, we found that greedy algorithms that usually have higher regret bounds or no bound at all do just as well as Thompson sampling for this use-case, because recently onboarded MHPs are eager to schedule referrals; but as time passes, failure to maintain constraints leads to their dissatisfaction and a drop in their "payout" aka scheduling rate. Also, Thompson sampling does not allow us to put constraints on the MHPs. The oracle, which is a supervised learning algorithm, uses context for new MHPs to predict their success rate quite well, and it reduced our concern for exploration and regret.

So, this problem can be simplified into the following:
- A system consists of sending m referrals to n MHPs
- An oracle gives likelihood for each referral-MHP pair
– MHPs have an inherent/true scheduling rate
- Each MHP has a maximum and minimum constraint on the number of referrals they can receive

The simplest solution to this problem could be to use a greedy algorithm for sending a referral to a MHP. For any given referral, we will send it to the MHP with the highest payout, but then, there is no
guarantee that it will satisfy the minimum and maximum constraints. Let's consider an example to understand the problem: 

#### Example:

Let's consider a provider MHP1 which is nearing its maximum quota/cap, but is not full yet. Let's say we can only send them two more referrals this week, but they end up being the top pick for four referrals. How do we decide which two to send?

Let's say the referrals, oracle scores, and MHPs look like this:
    
   |    |mhp1 | mhp2 | mhp3|
   |----|-----|------|-----|
   | r1 | 0.7 | 0.65 | 0.2 |
   | r2 | 0.8 | 0.7  | 0.75|
   | r3 | 0.9 | 0.2  | 0.1 |
   | r4 | 0.7 | 0.24 | 0.3 |

<b>Strategy 1 </b>

We suspend the MHP after the quota is met - after the first two referrals in this case. 
That means: 
- r1 and r2 are sent to MHP1 and 
- r3 is sent to MHP2 
- r4 is sent to MHP3
    
What are the expected number of appointments in this case? Assuming that the scores from the oracle are true probabilities.<br/>
Expected number of appointments = 0.7 + 0.8 + 0.2 + 0.3 = 2

<b>Strategy 2</b>

- r3 and r4 appointments to mhp1 instead
- r1 to mhp2
- r2 to mhp3
   
Expected number of appointments = 0.65 + 0.75 + 0.9 + 0.7 = 3 <br/>
So, a slightly different strategy resulted in one extra appointment in this group. This shows that a greedy approach might not be the best approach here.

In this case, we can easily see which two referrals to route to MHP1 because we can see all the referrals. Unfortunately, in production, the referrals are sent in real-time, as they come, and we have no information about the future. So, when deciding for r1, we do not know about r2, r3 and r4. We still want to try and optimize the total number of appointments. But how?

### Linear Optimization without Assignment
It is clear that we have reduced this to a standard linear optimization problem: it has an objective function (expected number of appointments), a cost function (scores) and constraints to satisfy. But there is one issue - we cannot send referrals in batches. They are sent in real-time as soon as a patient is received by the system.

This means that we cannot do "assignment" like a typical linear optimization problem. This notebook describes a solution that using linear optimization for this problem without assignment by using score adjustments 


### Formulation

#### Classic Binary Optimization

Let's assume we could batch referrals, what would the solution look like?

Let's say there are N MHPs, R referrals, this algorithm gives a probability score for each referral

Volume caps: $V_{0}, V_{1}, V_{2} ... V_{N}$

Boosting minimums $ B_{0}, B_{1}  ... B_{N}$ 

Let's consider the score matrix 



   | i\j |mhp_0| mhp_1|mhp_N| ... |
   |---- |-----|------|-----|-----|
   | r0  | 0.7 | 0.64 | 0.1 | ... |
   | r1  | 0.8 | 0.58  | 0.32| ... |
   | r2  | 0.91 | 0.35 | 0.3 | ... |
   | ... | ... | ...  | ... | ... |


For example, $s_{01} = 0.64$ , $s_{22} = 0.35$ 



Assignment Matrix:

   | i\j |mhp_0| mhp_1|mhp_N| ... |
   |---- |-----|------|-----|-----|
   | r0  |  0  |   1  |  0  | ... |
   | r1  |  1  |   0  |  0  | ... |
   | r2  |  0  |   0  |  0  | ... |
   | ... | ... | ...  | ... | ... |
   
<b>Formulation </b>:
    
$$ \max_{a} \sum \sum a_{ij} s_{ij}$$

    
$$ \sum_i a_{ij} < V_j  \qquad \forall  j $$

$$ \sum_i a_{ij} > B_j  \qquad \forall  j $$
    
$$ \sum_j a_{ij} = 1  \qquad \forall  i $$
    

So far so good, but this formulation is not very useful because we cannot send referrals to providers in batches. In the absence of assignments, we use compute score adjustments, which are listed below: 


#### Linear optimization w/o assignment

We force assignment to be on the maximum score per referral, which makes it a purely endogenous variable - as opposed to a decision variable. <br/>
We introduce a new set of decision variables $P$ (for score adjustments) such that $P_j$ is the score adjustment (boost + or penalty -) for each MHP $M_j$


   | j  |---> |      |     |     |
   |----|-----|------|-----|-----|
   | p  | 0.7 | 0.64 | 0.1 | ... |

<b>Formulation </b>:

$$ \max_{a} \sum \sum a_{ij} s_{ij}$$

    
$$ \sum_i a_{ij} < V_j  \qquad \forall  j $$
    
$$ \sum_j a_{ij} = 1  \qquad \forall  i $$

$$ s_{ij} + p_j \leq \sum_k a_{ik} (s_{ik} + p_k)  \qquad \forall  i \forall  j $$


There is a problem with the last equation. <br/>

It is not linear!


#### Convert Quadratic to Linear Using a Trick: McCormick Envelopes

$ z = xy $ 
where x is binary and y is continuous

$ L \leq y \leq U $ 
where L and U are bounds on y

Then the following constraints will make it linear

$$ z \leq Ux$$
$$ z \geq Lx$$
$$ z \leq y - L(1-x) $$
$$ z \geq y - U(1-x) $$


#### Final Formulation


$$ \max_{a} \sum \sum a_{ij} s_{ij} - (\sum abs (p_{j})) $$

    
$$ \sum_i a_{ij} < V_j  \qquad \forall  j $$
    
$$ \sum_j a_{ij} = 1  \qquad \forall  i $$

$$ s_{ij} + p_j \leq \sum_k a_{ik} s_{ik} + \sum_k z_{ik}   \qquad \forall  i  \forall  j$$


$$ z_{ik} = a_{ik} p_k $$

$$ -a_{ik} \leq z_{ik} \leq a_{ik} $$

$$ p_k - (1 - a_{ik}) \leq z_{ik} \leq p_k + (1 - a_{ik}) $$


That's it! <br/> Time to code ...


In [1]:
import os

from scipy.stats import pearsonr as pearson_corr

import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
%matplotlib inline
import pulp
from time import time


### Simulation

####  Generate Dummy Data

In [2]:
N_REFERRALS = 1500
N_MHPS = 10
MIN_, MAX_ = 0, None

# base scheduling rates 
mhp_names = ["mhp_%d" % i for i in range(N_MHPS)]
mhp_mu = np.random.uniform(0.1, 0.7, N_MHPS)
mhp_sigma = np.random.uniform(0.1, 0.4, N_MHPS) # standard deviation for probability scores
mhp_supply = np.random.uniform(0.2, 0.7, N_MHPS) # Number of nans

def generate_referral_data(mus, sigmas):
    df = pd.DataFrame({
        "mhp_%d" % i: np.random.normal(mus[i], sigmas[i], N_REFERRALS) for i in range(N_MHPS)
    })
    df = df[df >= 0].fillna(0)  # Replace negative values with zeros
    df[df > 1] = 1.0  # Replace values greater than 1.0 with 1.0
    # Introduce nulls because not all provideres are always eligible
    for i, col in enumerate(df.columns):
        frac = mhp_supply[i]
        df.loc[df.sample(frac=frac).index, col] = np.nan

    return df


def generate_maximums():
    mean_cap = int(N_REFERRALS/(N_MHPS)*0.6)
    caps = np.random.normal(mean_cap, int(mean_cap/2), N_MHPS)
    caps = [int(max(10, x)) for x in caps]
    caps[0] = MAX_  # first two providers are not capped
    caps[1] = MAX_
    return caps


def generate_minimums(maximums):
    # randomly pick providers to be boosted with about 40 % probability
    # ** TURNING OFF MINIMUMS JUST TO COMPARE MAXIMUMS TO A SIMPLE GREEDY ALGORITHM
    boosted = (np.random.uniform(0, 1, N_MHPS) > 1.0).astype(int)
    mins = [int(maximums[i] / 2.0) if (boosted[i] and maximums[i] is not None)
            else 0 for i in range(N_MHPS)]
    mins[0] = None  # first two providers are hardcoded for control
    mins[1] = 0
    return mins
    
df = generate_referral_data(mhp_mu, mhp_sigma)
maximums = generate_maximums()
minimums = generate_minimums(maximums)

df.head()

,mhp_0,mhp_1,mhp_2,mhp_3,mhp_4,mhp_5,mhp_6,mhp_7,mhp_8,mhp_9
0,0.195125,0.026184,0.438068,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
1,0.192366,NaN,0.000000,0.540930,NaN,NaN,NaN,0.280652,0.469458,NaN
2,NaN,0.117338,NaN,NaN,NaN,0.325129,0.13247,0.336539,NaN,NaN
3,0.208126,NaN,NaN,0.461050,0.219322,NaN,NaN,0.794607,NaN,NaN
4,NaN,NaN,0.374998,0.293219,0.315532,0.317780,NaN,0.724434,0.825048,NaN


####  Dummy Data with Maximums and Minimums

In this section, we demonstrate the total referrals that an MHP would receive with a greedy strategy.
- "current_referrals" is the count the MHP would get with greedy
- "minimum" and "maximums" are the constraints set on them

In [3]:
df_vol = pd.DataFrame({
    "mean_score": df.mean().round(2),
    "std_dev_score": df.std().round(2),
    "supply_shortage": mhp_supply.round(2),
    "current_referrals": df.idxmax(axis=1).value_counts(),
    "minimums": minimums,
    "maximums": maximums,
    }, index=mhp_names)  # .fillna(0)

df_vol['current_referrals'] = df_vol.current_referrals.fillna(0).astype(int)
df_vol

,mean_score,std_dev_score,supply_shortage,current_referrals,minimums,maximums
mhp_0,0.20,0.17,0.55,15,NaN,NaN
mhp_1,0.23,0.26,0.53,58,0.0,NaN
mhp_2,0.17,0.16,0.29,16,0.0,45.0
mhp_3,0.32,0.13,0.23,53,0.0,163.0
mhp_4,0.41,0.31,0.46,181,0.0,10.0
mhp_5,0.57,0.22,0.33,362,0.0,60.0
mhp_6,0.22,0.23,0.66,32,0.0,107.0
mhp_7,0.52,0.26,0.34,288,0.0,88.0
mhp_8,0.69,0.18,0.58,289,0.0,37.0
mhp_9,0.51,0.29,0.51,206,0.0,179.0


#### Optimization Function

We use Pulp to code the optimizer formulated above.

In [4]:
def run_linear_optimization(df, maximums, minimums):
    N_REFERRALS, N_MHPS = df.shape
    st = time()
    
    scores = df.values
    prob = pulp.LpProblem("Network Optimization", pulp.LpMaximize)

    adjustment = pulp.LpVariable.dicts("adjustment", range(N_MHPS), lowBound=-1, upBound=1, cat='Continuous')
    adj_abs = pulp.LpVariable.dicts("adjustment_absolute", range(N_MHPS), lowBound=0, upBound=1, cat='Continuous')
    
    z_dummy = pulp.LpVariable.dicts("z_dummy", [(i, j) for i in range(N_REFERRALS) for j in range(N_MHPS)], -1, 1, cat='Continuous')

    assignment = pulp.LpVariable.dicts("assignment", [(i,j) for i in range(N_REFERRALS) for j in range(N_MHPS)], 0, 1, cat='Integer')

    # The objective function is added to 'prob' first
    prob += pulp.lpSum([assignment[i, j]*scores[i, j] for i in range(N_REFERRALS) for j in range(N_MHPS) if pd.notnull(scores[i, j])]) , "expected_appts"
    
    # Constraints
    # 1. Only one MHP assignment per referral 
    for i in range(N_REFERRALS):
        prob += pulp.lpSum([assignment[i, j] for j in range(N_MHPS) if pd.notnull(scores[i, j])]) == 1.0, "assignment_sum_%d" % i

    # 2. Volume caps
    for j in range(N_MHPS):
        if pd.notnull(maximums[j]):
            prob += pulp.lpSum([assignment[i, j] for i in range(N_REFERRALS) if pd.notnull(scores[i, j])]) <= maximums[j]
        
    # 3. Boosted mins
    for j in range(N_MHPS):
        n_referrals_for_mhp = (scores[:, j] >= 0).sum()
        if pd.notnull(minimums[j]) and (n_referrals_for_mhp > 0):
            # find the minimum referrals they are eligible for            
            lower_bound = min(minimums[j], n_referrals_for_mhp)
            if pd.notnull(maximums[j]):  # lower_bound should not be too close to upper bound
                lower_bound = min(lower_bound, int(maximums[j]*0.7))
            prob += pulp.lpSum([assignment[i, j] for i in range(N_REFERRALS) if pd.notnull(scores[i, j])]) >= lower_bound        

    for j in range(N_MHPS):
        # adjustment[j].setInitialValue(0)
        if pd.isnull(minimums[j]):
            adjustment[j].upBound = 0
            
        if pd.isnull(maximums[j]):
            adjustment[j].lowBound = 0

        # no constraints for MHPs without limits
        if pd.isnull(minimums[j]) and pd.isnull(maximums[j]):
            adj_abs[j].upBound = 0
            adj_abs[j].lowBound = 0
            
    for i in range(N_REFERRALS):
        for j in range(N_MHPS):
            if pd.isnull(scores[i, j]):
                assignment[i, j].lowBound = 0
                assignment[i, j].upBound = 0
                z_dummy[i, j].lowBound = 0
                z_dummy[i, j].upBound = 0
                
            if pd.isnull(minimums[j]) and pd.isnull(maximums[j]): # no constraints for MHPs without limits
                z_dummy[i, j].lowBound = 0
                z_dummy[i, j].upBound = 0
                
    # 4. Assignment should only happen to the maximum score after applying the adjustment
    for i in range(N_REFERRALS):
                 
        # Trick to turn quadratic (assignment * adjustment) into Linear
        for k in range(N_MHPS): 
            U = 1  # 0 if minimums[k] is None else 1
            L = -1 # 0 if maximums[k] is None else -1
            
            if pd.isnull(scores[i, k]):
                continue
            if pd.isnull(minimums[k]) and pd.isnull(maximums[k]): # no constraints for MHPs without limits
                continue               
                
            prob += z_dummy[i, k] <= U * assignment[i, k]
            prob += z_dummy[i, k] >= L * assignment[i, k]
            prob += z_dummy[i, k] <= adjustment[k] - L *(1 - assignment[i, k])
            prob += z_dummy[i, k] >= adjustment[k] - U * (1 - assignment[i, k])

        for j in range(N_MHPS):                
            # <= (pulp.lpSum([assignment[i, k] * (scores[i, k] + adjustment[j]) for k in range(N_MHPS)]) 
            # Ideally you want this ^ , but it is quadratic, so introducing a dummy variable z_dummy
            if pd.isnull(scores[i, j]):
                continue
            prob += (scores[i, j] + adjustment[j]) <= (pulp.lpSum([assignment[i, k] * scores[i, k] 
                                                                   for k in range(N_MHPS) if pd.notnull(scores[i, k])]) +
                                                       pulp.lpSum([z_dummy[i, k] for k in range(N_MHPS)]))
            
    
    for j in range(N_MHPS):
        prob += adjustment[j] <= adj_abs[j]
        prob += -adjustment[j] <= adj_abs[j]

    print("Optimization Initialized in %.2f seconds" % (time() - st))
    print("Running solver ...")
    pulp.list_solvers(onlyAvailable=True)
    prob.solve(pulp.PULP_CBC_CMD(gapRel=0.1, timeLimit=600))
    # prob.solve(pulp.PULP_CBC_CMD(timeLimit=10, msg=1, gapRel=0.1, logPath="/tmp/pulp.log"))
    # prob.solve(pulp.CPLEX_PY())
    print("Solver status:", pulp.LpStatus[prob.status])
    print("Solver status:", (pulp.LpStatus[prob.status] == 'Optimal'))

    print("Total time: %.2f seconds" % (time() - st))
    print("Value of objective: %.2f" % pulp.value(prob.objective))
    columns = df.columns.to_list()
    adjustment = pd.Series({columns[j]: adjustment[j].varValue for j in range(N_MHPS)})
    return adjustment, assignment


#### Run Simulation

Let's send the referral data through the optimizer and look at the results

In [5]:
# minimums = [None] * len(maximums)
adjustment, assignment = run_linear_optimization(df, maximums, minimums)
assign = ["mhp_%d" % j  for i in range(N_REFERRALS) for j in range(N_MHPS) if assignment[i, j].varValue == 1]
df_vol["new_referrals_using_assignment"] = pd.value_counts(assign)
# df_vol['adjustment'] = pd.Series({"mhp_%d" % j: adjustment[j].varValue for j in range(N_MHPS)})

df_vol['adjustment'] = adjustment
df_new = df.copy(deep=True)

for i in range(N_MHPS):
    cc = "mhp_%d" % i
    df_new[cc] = (df_new[cc] + df_vol['adjustment'].iloc[i])
    
# Note - the new counts of referrals using adjustment should be exactly the same
# as the counts with assignment (decision variable), because assignment of a referral 
# happens to the MHP with maximum `score + adjustment` for the given referral


# df_vol['new_referrals_using_adjustment'] = df_new.idxmax(axis=1).value_counts()
# df_vol[['minimums', 'maximums', 'current_referrals', 'new_referrals_using_assignment', 'adjustment']]

/home/rdalvi/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1199: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Optimization Initialized in 2.73 seconds
Running solver ...
Solver status: Optimal
Solver status: True
Total time: 87.27 seconds
Value of objective: 711.55


In [6]:
df_vol

,mean_score,std_dev_score,supply_shortage,current_referrals,minimums,maximums,new_referrals_using_assignment,adjustment
mhp_0,0.20,0.17,0.55,15,NaN,NaN,404,0.000000
mhp_1,0.23,0.26,0.53,58,0.0,NaN,407,0.000901
mhp_2,0.17,0.16,0.29,16,0.0,45.0,45,-0.448756
mhp_3,0.32,0.13,0.23,53,0.0,163.0,163,-0.389212
mhp_4,0.41,0.31,0.46,181,0.0,10.0,10,-1.000000
mhp_5,0.57,0.22,0.33,362,0.0,60.0,60,-0.856634
mhp_6,0.22,0.23,0.66,32,0.0,107.0,107,-0.290525
mhp_7,0.52,0.26,0.34,288,0.0,88.0,88,-0.776431
mhp_8,0.69,0.18,0.58,289,0.0,37.0,37,-0.887036
mhp_9,0.51,0.29,0.51,206,0.0,179.0,179,-0.605487


*The optimizer works !*

As we can see here, the best MHPs (high scheduling rate) got a lot of referrals, and were maxed out when possible.
On the other hand, the MHPs with the lowest scheduling rate did not get many referrals.

### Lift Over Greedy Algorithm

#### Expected Number of Appointments of the Linear Optimizer

In [7]:
def get_expected_appts_linop(df):
    df_new = df.copy(deep=True)

    for i in range(N_MHPS):
        cc = "mhp_%d" % i
        df_new[cc] = (df_new[cc] + df_vol['adjustment'].iloc[i])

    expected_appts = 0
    for i in range(N_REFERRALS):
        row = df_new.iloc[i]
        best_mhp = row.idxmax()
        best_score = df.loc[i, best_mhp]  # Note - this is the original df
        expected_appts += best_score
        
    print("Expected number of appointments Linop: %d" % expected_appts)
    mhp_counts = df_new.idxmax(axis=1).value_counts()
    return mhp_counts
    
_ = get_expected_appts_linop(df)

Expected number of appointments Linop: 712


#### Expected Number of Appointments in Greedy Algorithm

In [8]:
def get_expected_appts_greedy(df):
    df_new = df.copy(deep=True)

    referral_counts = {"mhp_%d" % i: 0 for i in range(N_MHPS)}
    expected_appts = 0
    greedy_assignments = []
    overdraft = 0
    for i in range(N_REFERRALS):        
        row = df_new.iloc[i]
        best_mhp = row.idxmax()
        if pd.isnull(best_mhp):  # No MHPs available
            best_mhp = df.iloc[i].idxmax()  # original df
            best_score = df.loc[i, best_mhp]  # original df
        else:
            best_score = df_new.loc[i, best_mhp]

        referral_counts[best_mhp] += 1  # maximum cannot be zero                
        expected_appts += best_score
        greedy_assignments.append(best_mhp)
        max_for_best_mhp = df_vol.maximums.loc[best_mhp]
        
        if (best_mhp in df_new.columns.tolist()) and \
            referral_counts[best_mhp] >= max_for_best_mhp:
            del df_new[best_mhp]   # MHP maxed out, remove it 
        

    print("Expected number of appointments Greedy: %d" % expected_appts)
    print("Expected number of maximum possible appointments: %d" % df.max(axis=1).sum())
    mhp_counts = pd.Series(greedy_assignments).value_counts()
    return mhp_counts
    
_ = get_expected_appts_greedy(df)

Expected number of appointments Greedy: 710
Expected number of maximum possible appointments: 1117


### Test on Unseen Data with Noise

In [12]:
noise_mu = 0
noise_sigma1 = 0.001
noise_sigma2 = 0.001

mhp_mu_new = [x + np.random.normal(noise_mu, noise_sigma1) for x in mhp_mu]
mhp_sigma_new = [x + np.random.normal(noise_mu, noise_sigma2) for x in mhp_sigma]

df_test = generate_referral_data(mhp_mu_new, mhp_sigma_new)
linop_counts = get_expected_appts_linop(df_test)
greedy_counts = get_expected_appts_greedy(df_test)

df_test_vol = pd.DataFrame({
    "mean_score": df.mean().round(2),
    "std_dev_score": df.std().round(2),
    "current_referrals": df.idxmax(axis=1).value_counts(),
    "minimums": minimums,
    "maximums": maximums,
    }, index=mhp_names)  # .fillna(0)

df_test_vol['current_referrals'] = df_vol.current_referrals.fillna(0).astype(int)
df_test_vol['linop_counts'] = linop_counts
df_test_vol['greedy_counts'] = greedy_counts

x = (df_test_vol['linop_counts'] - df_test_vol.maximums) 
print("Overdraft linop: %.2f" % x[x>0].sum())

x = (df_test_vol['greedy_counts'] - df_test_vol.maximums) 
print("Overdraft greedy: %.2f" % x[x>0].sum())

Expected number of appointments Linop: 727
Expected number of appointments Greedy: 707
Expected number of maximum possible appointments: 1114
Overdraft linop: 52.00
Overdraft greedy: 192.00


### Conclusion

**The linear optimization solution not only has a higher number of appointments than greedy, but also a lower overdraft**. 
We are counting the score of overdrafted referrals towards the expected number of appointments, which is not true in practice. The greedy algorithm has 707 referrals with 192 overdrafted. Since overdrafted referrals have a high chance of rejection, the actual number of appointments is much lower. If we assume the chance of success for an overdrafted referral to be zero, the total appointments for greedy would be about 600, compared to ~700 for linear optimizer, giving over 15% improvement.

**This demo was limited to maximum constraints, where we could demonstrate a comparison to a greedy algorithm.
When we turn on minimum constraints, the linear optimizer gives even better results than greedy**

Accross many trials of this experiment, linear optimizer consistently performs better.